In [44]:
import numpy as np
import cv2
import heapq

In [45]:
def scale(input):
    return input*2

In [46]:
h = scale(50) #50 mm scaled up
w = scale(180)



map = np.zeros((h, w, 3), dtype=np.uint8)

clearance = scale(5)
radius = scale(5)
buffer = clearance+radius

map_limits = map.copy()
map_limits = cv2.cvtColor(map_limits, cv2.COLOR_BGR2GRAY)
map_limits = cv2.rectangle(map_limits, (0,0), (w,h), (255), 1)
locations = np.where(map_limits == (255))

angle_list = []
for i in range(0,12):
    angle_list.append(i*30)

boundary = set()
for i in range(0,len(locations[0])):
    for a in angle_list:
        boundary.add((locations[0][i], locations[1][i], a))


del map_limits
del locations


In [47]:
def add_boundary(inpt_x,inpt_y):
    # add given point to the boundary sets
    
    for a in angle_list:
        boundary.add((inpt_x,inpt_y,a))

    # add all points in a box around given point to boundary sets
    for x in range(inpt_x-buffer, inpt_x+(buffer+1)):
        for y in range(inpt_y-buffer, inpt_y+(buffer+1)):
            for a in angle_list:
                boundary.add((x,y,a))

In [48]:
# define obstacles
for y in range(scale(17),scale(35)):
    # define obstacle E
    for x in range(scale(20),scale(40)):
        if scale(16) < y <= scale(20):
            if scale(21) < x <= scale(39):
                # top horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if scale(20) < y <= scale(23):
            if scale(21) < x <= scale(25):
                # between top and middle horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if scale(23) < y <= scale(27):
            if scale(21) < x <= scale(39):
                # middle horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if scale(27) < y <= scale(30):
            if scale(21) < x <= scale(25):
                # between middle and bottom horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if scale(30) < y <= scale(34):
            if scale(21) < x <= scale(39):
                #bottom horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle N
    for x in range(scale(40),scale(60)):
        if scale(41) < x <= scale(45):
            # first vertical of N
            map[y,x] = (255,0,0)
            add_boundary(x,y)

            # second vertical of N
        if scale(55) < x <= scale(59):
            map[y,x] = (255,0,0)
            add_boundary(x,y)

            # diagonal of N determined by equations of two lines
        if scale(45) < x <= scale(55):
            if ((1.2*x)-scale(38))< y <= ((1.2*x)-scale(32)):
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle P
    for x in range(scale(60), scale(80)):

        # main vertical of P
        if scale(61) < x <= scale(67):
            map[y,x] = (255,0,0)
            add_boundary(x,y)

        # round part of P determined by ellipse equation
        if scale(67) < x <= scale(79):
            if (((x-scale(67))**2)/((scale(12))**2))+(((y-scale(22))**2)/((scale(6))**2)) <= 1:
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle M
    for x in range(scale(80),scale(100)):

        # first vertical of M
        if scale(81) < x <= scale(85):
            map[y,x] = (255,0,0)
            add_boundary(x,y)
        
        # first angle of M determined by equations of two lines
        if scale(85) < x <= scale(90):
            if (((9/5)*x)-scale(137)) < y <= (((9/5)*x)-scale(129)):
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        # second angle of M determined by equations of two lines
        if scale(90) < x <= scale(95):
            if (((-9/5)*x)+scale(187)) < y <= (((-9/5)*x)+scale(195)):
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        
        # last vertical of M 
        if scale(95)< x <= scale(99):
            map[y,x] = (255,0,0)
            add_boundary(x,y)
    
    # define obstacle 6
    for x in range(scale(100),scale(120)):

        # draw main circle per equation
        if ((x-scale(110))**2)+(y-scale(28.5))**2 <= (scale(5.5))**2:
            # add both 6's spaced 20 pixels apart
            map[y,x] = (255,0,0)
            map[y,x+scale(20)] = (255,0,0)
            add_boundary(x,y)
            add_boundary(x+scale(20),y)

        # draw second circle at tip of "tail" of 6
        if ((x-scale(110.85))**2)+(y-scale(18))**2 <= (scale(2))**2:
            map[y,x] = (255,0,0)
            map[y,x+scale(20)] = (255,0,0)
            add_boundary(x,y)
            add_boundary(x+scale(20),y)
        
        # draw part of a ring defined by two circles and two lines
        # to define the "tail" of 6
        if x-scale(92.85) < y <= scale(28.5):
            if (scale(12.85))**2 < (((x-scale(121.35))**2)+((y-scale(28.5))**2)) <= (scale(16.85))**2:
                map[y,x] = (255,0,0)
                map[y,x+scale(20)] = (255,0,0)
                add_boundary(x,y)
                add_boundary(x+scale(20),y)

    # define obstacle 1
    for x in range(scale(140),scale(160)):
        if scale(16) < y <= scale(20.62):
            if scale(144) < x <= scale(152):
                # top horizontal of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        if scale(20.62) < y <= scale(30):
            if scale(148) < x <= scale(152):
                # main vertical of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        if scale(30) < y <= scale(34):
            if scale(144) < x < scale(156):
                # bottom horizontal of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)

In [49]:
start_x = scale(5)
start_y = scale(5)
start_t = 0
start = (start_x,start_y,start_t)

end_x = scale(100)
end_y = scale(48)
end_t = 0
end = (end_x,end_y,end_t)
step_size = scale(3)


In [50]:
open_list = []
closed_list = []

heapq.heapify(closed_list)
heapq.heapify(open_list)

def distance(p1,p2):
    dist = np.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2))
    dist = int(dist)
    return dist

start_ctg = distance(start,end)
heapq.heappush(open_list, (start_ctg, 0, start_ctg, start, start))

In [51]:
def move(node,angle):
    p_ctc = node[1]
    p_x = node[4][0]
    p_y = node[4][1]
    c_x = int(p_x+(step_size*np.cos(np.deg2rad(angle))))
    c_y = int(p_y+(step_size*np.sin(np.deg2rad(angle))))
    c_t = angle
    p_coord = node[4]
    c_coord = (c_x, c_y, c_t)
    c_xy = (c_x, c_y)
    c_ctc = p_ctc+step_size
    c_ctg = distance(c_coord,end)
    c_tot = c_ctc+c_ctg
    if c_coord in boundary:
        return
    for item in open_list:
        item_xy = (item[4][0],item[4][1])
        item_t = item[4][2]
        if distance(c_xy,item_xy) <= scale(0.5) and item_t == c_t:
            if item[0] > c_tot:
                open_list.remove(item)
            else:
                return
    heapq.heappush(open_list, (c_tot, c_ctc, c_ctg, p_coord, c_coord))
    return


In [53]:
dist_to_goal = start_ctg
exy = (end[0],end[1])

while True:
    
    parent_node = heapq.heappop(open_list)
    boundary.add(parent_node[4])
    heapq.heappush(closed_list, parent_node)
    pxy = (parent_node[4][0],parent_node[4][1])
    dist_to_goal = distance(pxy,exy)
    

    # print(dist_to_goal)

    for item in [-2, -1, 0, 1, 2]:
        ang = parent_node[4][2] + (30*item)
        if ang >= 360:
            ang = ang-360
        if ang < 0:
            ang = 360 - (-1*ang)
        move(parent_node,ang)
    
    if dist_to_goal < scale(1.5):
        if parent_node[4][2] == end[2]:
            break

KeyboardInterrupt: 

In [54]:
closed_list

[(208, 0, 208, (10, 10, 0), (10, 10, 0)),
 (208, 6, 202, (10, 10, 0), (15, 13, 30)),
 (208, 12, 196, (15, 13, 30), (20, 16, 30)),
 (209, 6, 203, (10, 10, 0), (13, 15, 60)),
 (209, 6, 203, (10, 10, 0), (16, 10, 0)),
 (209, 12, 197, (15, 13, 30), (21, 13, 0)),
 (209, 12, 197, (16, 10, 0), (21, 13, 30)),
 (209, 12, 197, (16, 10, 0), (22, 10, 0)),
 (209, 18, 191, (21, 13, 0), (27, 13, 0)),
 (209, 18, 191, (22, 10, 0), (27, 13, 30)),
 (210, 12, 198, (13, 15, 60), (18, 18, 30)),
 (210, 12, 198, (13, 15, 60), (19, 15, 0)),
 (210, 12, 198, (15, 13, 30), (18, 18, 60)),
 (210, 12, 198, (16, 10, 0), (19, 15, 60)),
 (210, 18, 192, (22, 10, 0), (28, 10, 0)),
 (210, 24, 186, (27, 13, 0), (33, 13, 0)),
 (210, 24, 186, (28, 10, 0), (33, 13, 30)),
 (210, 24, 186, (28, 10, 0), (34, 10, 0)),
 (211, 6, 205, (10, 10, 0), (15, 7, -30)),
 (211, 12, 199, (13, 15, 60), (16, 20, 60)),
 (211, 12, 199, (15, 7, -30), (20, 10, 30)),
 (211, 12, 199, (15, 7, -30), (21, 7, 0)),
 (211, 12, 199, (15, 13, 30), (20, 10, -

In [ ]:
final_path = []
path_node = closed_list[-1][4]
while path_node != start:
    for item in closed_list:
        if item[4] == path_node: 
            final_path.append(item)
            path_node = item[3]

final_path.sort() # reorder path to go from start to finish


In [ ]:
map_display = map.copy()

for item in closed_list:
    par_xy = (item[3][0],item[3][1])
    chi_xy = (item[4][0],item[4][1])
    cv2.line(map_display,par_xy,chi_xy,(255,255,255),1)
    frame = map_display.copy()
    cv2.imshow('animation',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# for item in final_path:
#     par_xy = (item[3][0],item[3][1])
#     chi_xy = (item[4][0],item[4][1])
#     cv2.line(map_display,par_xy,chi_xy,(255,0,0),1)
#     frame = map_display.copy()
#     frame = cv2.resize(frame,(0,0), fx=3, fy=3)
#     cv2.imshow('animation',frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow("map",map)
cv2.waitKey(0)
cv2.destroyAllWindows()